In [1]:
import pandas as pd
from glob import glob
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
ROOT = "evals"

In [5]:
numbers, genes =[], []
for file in glob("{}/*/report/*.csv".format(ROOT)):
    base_name = file.split("/")[1]
    family, evalue = base_name.split("_")
#     family = family + "_" + blasts
    if "numbers_clean" in file:
        number = pd.read_csv(file)
        number.loc[:, 'family'] = family
        number.loc[:, 'evalue'] = evalue
        numbers.append(number)
    elif "genes" in file:
        gene = pd.read_csv(file)
        gene.loc[:, "family"] = family
        gene.loc[:, "Evalue"] = evalue
        gene = gene.loc[~(gene['cluster'].isin(['na', '0',"0.0"]))]
        genes.append(gene)
numbers_table = pd.concat(numbers)
genes_table = pd.concat(genes)

In [6]:
number_group = numbers_table.groupby(['species','family','evalue'])

In [7]:
total_paralogs = number_group.sum()['paralogs']

In [8]:
number_clusters = number_group.count()['cluster']

In [9]:
cluster_paralogs = pd.concat([total_paralogs, number_clusters],1)

cluster_paralogs.columns = ["Duplicates", "Clusters"]

In [10]:
sns.set_style("white")
for sp in set(cluster_paralogs.index.get_level_values(0)):
    print(sp)
    for value_plot in ["Duplicates", "Clusters"]:
        sp_title = sp.replace("_", " ")
        sp_tab = cluster_paralogs.loc[sp]
        evalues = ['1','1e-1','1e-2','1e-3','1e-4','1e-5','1e-6','1e-7','1e-8','1e-9','1e-10']
        barplot = sns.barplot(x="family",y=value_plot,hue='evalue',data=sp_tab.reset_index(),
                              hue_order=evalues, palette=sns.color_palette("gray",n_colors=11))
        barplot.legend(loc=(1,0),title='Evalue',ncol=1)
        sns.despine()
        plt.title(sp_title)
        plt.ylabel(value_plot)
        plt.setp(barplot.get_xticklabels(), rotation=90);
        plt.tight_layout(rect=(0,0,0.85,1))
        plt.savefig("/home/alekos/Dropbox/placenta_clusters_paper/revision/figures/5000/{}_{}.svg".format(sp,value_plot))
        plt.close()

Papio_anubis
Capra_hircus
Pongo_abelii
Microtus_ochrogaster
Canis_lupus_familiaris
Rattus_norvegicus
Mus_musculus
Sus_scrofa
Bos_taurus
Macaca_fascicularis
Homo_sapiens
Ovis_aries
Gorilla_gorilla_gorilla
Monodelphis_domestica
Ornithorhynchus_anatinus
Oryctolagus_cuniculus
Callithrix_jacchus
Chlorocebus_sabaeus
Pan_troglodytes
Felis_catus
Nomascus_leucogenys
Equus_caballus
Macaca_mulatta


In [11]:
xls = pd.ExcelWriter("/home/alekos/Dropbox/placenta_clusters_paper/revision/evalues_5000.xlsx")
for sp in set(cluster_paralogs.index.get_level_values(0)):
    cluster_paralogs.loc[sp].to_excel(xls, sp)
    xls.save()